Random forest preparation:
Below is the code I used for the IMD data for the FBA c/w - will be different as I am doing a classifiction rather than a regression and will use scikit rather than sklearn

Also, for the FBA, there were 7 input features (OSM info) and 1 output (IMD score). For this there will be up to 358 input features (spectral bands) and 1 output (PFT). Therefore will need a table with all of the info in - make by creating a vector file of the bsq? - test this.

Want to iterate through the data rather than adding manually (358 bands)

Then need to split into X (all the bands) and y (PFT)

When complete, try a range of proportions of training and test data and remember to cross validate results.

Also possibly use the k-means clustering to validate and compare results

In [4]:
import pandas as pd
import geopandas
cross_lochs = pd.read_csv('cross_lochs_sample_for_rf2.csv')

cross_lochs['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs['geometry'])

cross_lochs_gdf = geopandas.GeoDataFrame(cross_lochs, geometry='geometry')

cross_lochs_gdf


,Unnamed: 0,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT,line
0,617,0.000951,0.005150,0.005988,0.006720,0.007361,0.010606,0.012082,0.014234,0.015797,...,0.097626,0.096350,0.095506,0.098056,0.096859,618,POINT (444764.308 6472590.655),0.000951,calluna_mix,4
1,822,0.002991,0.004075,0.004819,0.005989,0.007388,0.009729,0.012035,0.013280,0.016029,...,0.091884,0.090096,0.090060,0.089917,0.090350,823,POINT (444774.308 6472585.655),0.002991,calluna_mix,5
2,1230,0.002812,0.003526,0.005374,0.006186,0.007536,0.009726,0.011938,0.013003,0.016773,...,0.087527,0.090578,0.089190,0.087724,0.090253,1231,POINT (444784.308 6472575.655),0.002812,calluna_mix,7
3,1841,0.002776,0.004215,0.004849,0.006305,0.008160,0.010193,0.012259,0.014886,0.015938,...,0.093998,0.096585,0.099090,0.097692,0.098034,1842,POINT (444794.308 6472560.655),0.002776,calluna_mix,10
4,2452,0.003292,0.003483,0.003686,0.005693,0.007814,0.009010,0.012300,0.013145,0.015174,...,0.094432,0.094272,0.097161,0.097323,0.095702,2453,POINT (444804.308 6472545.655),0.003292,calluna_mix,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,19455,0.000811,0.001748,0.002514,0.003086,0.002629,0.003547,0.005152,0.006167,0.008431,...,0.010550,0.012407,0.010226,0.010765,0.010614,19456,POINT (445574.308 6472130.655),0.000811,water,96
176,21481,0.000730,0.001129,0.001588,0.002542,0.002014,0.003243,0.004675,0.004775,0.006566,...,0.005632,0.006227,0.007418,0.005301,0.004491,21482,POINT (445554.308 6472080.655),0.000730,water,106
177,22294,0.000828,0.001481,0.002596,0.002506,0.002702,0.003921,0.005196,0.006644,0.008155,...,0.010491,0.014097,0.011939,0.011159,0.008924,22295,POINT (445559.308 6472060.655),0.000828,water,110
178,41439,0.000852,0.001206,0.003087,0.003662,0.005048,0.007325,0.009386,0.011490,0.013876,...,0.080829,0.080322,0.081170,0.080854,0.080922,41440,POINT (444859.308 6471585.655),0.000852,calluna_mix,205


In [5]:
# cross_lochs_vis = cross_lochs_gdf[cross_lochs_gdf.columns[7:66],[359:363]] # issues with the second set of square brackets

cross_lochs_vis_bands = cross_lochs_gdf.iloc[:,6:66]
cross_lochs_RE_bands = cross_lochs_gdf.iloc[:,62:76]
cross_lochs_NIR_bands = cross_lochs_gdf.iloc[:,66:186]
cross_lochs_SWIR_bands = cross_lochs_gdf.iloc[:,186:359]

cross_lochs_general = cross_lochs_gdf.iloc[:,359:363]

cross_lochs_vis = pd.concat([cross_lochs_vis_bands, cross_lochs_general], axis=1)
cross_lochs_RE = pd.concat([cross_lochs_RE_bands, cross_lochs_general], axis=1)
cross_lochs_NIR = pd.concat([cross_lochs_NIR_bands, cross_lochs_general], axis = 1)
cross_lochs_SWIR = pd.concat([cross_lochs_SWIR_bands, cross_lochs_general], axis = 1)


cross_lochs_RE.head()

,band62,band63,band64,band65,band66,band67,band68,band69,band70,band71,band72,band73,band74,band75,POINTID,geometry,GRID_CODE,PFT
0,0.078536,0.085416,0.095944,0.107573,0.120429,0.133712,0.146125,0.157834,0.169158,0.179111,0.188062,0.195035,0.200163,0.205071,618,POINT (444764.308 6472590.655),0.000951,calluna_mix
1,0.076062,0.082774,0.092966,0.105358,0.118332,0.130206,0.141944,0.153875,0.163983,0.172715,0.181595,0.188657,0.193181,0.198372,823,POINT (444774.308 6472585.655),0.002991,calluna_mix
2,0.074502,0.081670,0.090580,0.101656,0.114236,0.124746,0.136281,0.148828,0.158416,0.166149,0.175543,0.182028,0.187960,0.191522,1231,POINT (444784.308 6472575.655),0.002812,calluna_mix
3,0.075528,0.082384,0.090955,0.100739,0.111517,0.121905,0.132367,0.141862,0.150342,0.159306,0.166974,0.172433,0.177887,0.181061,1842,POINT (444794.308 6472560.655),0.002776,calluna_mix
4,0.078022,0.085424,0.097117,0.108545,0.121420,0.134342,0.145877,0.158683,0.168526,0.177929,0.187749,0.195050,0.201413,0.204711,2453,POINT (444804.308 6472545.655),0.003292,calluna_mix


In [7]:
from sklearn.model_selection import train_test_split

train_shrub_sphagnum, test_shrub_sphagnum = train_test_split(cross_lochs_RE[cross_lochs_RE['PFT'].str.contains('shrub_sphagnum')], test_size=0.3)
train_water, test_water = train_test_split(cross_lochs_RE[cross_lochs_RE['PFT'].str.contains('water')], test_size=0.3)
train_sphagnum_r, test_sphagnum_r = train_test_split(cross_lochs_RE[cross_lochs_RE['PFT'].str.contains('sphagnum_r')], test_size=0.3)
train_pool_bogbean, test_pool_bogbean = train_test_split(cross_lochs_RE[cross_lochs_RE['PFT'].str.contains('pool_bogbean')], test_size=0.3)
train_calluna_mix, test_calluna_mix = train_test_split(cross_lochs_RE[cross_lochs_RE['PFT'].str.contains('calluna_mix')], test_size=0.3)
train_rushes_sedges, test_rushes_sedges = train_test_split(cross_lochs_RE[cross_lochs_RE['PFT'].str.contains('rushes_sedges')], test_size=0.3)

train_frames = [train_shrub_sphagnum, train_water, train_sphagnum_r, train_pool_bogbean, train_calluna_mix, train_rushes_sedges]
test_frames = [test_shrub_sphagnum, test_water, test_sphagnum_r, test_pool_bogbean, test_calluna_mix, test_rushes_sedges]


train = pd.concat(train_frames)
test = pd.concat(test_frames)


train.head()


,band62,band63,band64,band65,band66,band67,band68,band69,band70,band71,band72,band73,band74,band75,POINTID,geometry,GRID_CODE,PFT
99,0.098862,0.105970,0.117665,0.129028,0.138074,0.150081,0.158953,0.170322,0.176225,0.183672,0.191324,0.198992,0.202711,0.206783,19341,POINT (444999.308 6472130.655),0.001566,shrub_sphagnum
105,0.079250,0.087100,0.097352,0.109018,0.119823,0.130280,0.139682,0.149977,0.158758,0.163870,0.170214,0.176583,0.181596,0.182750,24626,POINT (445034.308 6472000.655),0.000598,shrub_sphagnum
95,0.073791,0.081639,0.091057,0.103281,0.115778,0.128240,0.138248,0.149836,0.160428,0.167386,0.177859,0.185074,0.188518,0.193041,13870,POINT (445049.308 6472265.655),0.001928,shrub_sphagnum
103,0.076728,0.083052,0.092856,0.103407,0.114569,0.123158,0.132884,0.141585,0.148989,0.156036,0.162252,0.168183,0.172545,0.174828,23861,POINT (445269.308 6472020.655),0.001278,shrub_sphagnum
117,0.067387,0.074295,0.082307,0.094084,0.105012,0.115725,0.125704,0.137779,0.144590,0.154660,0.165633,0.169904,0.173408,0.175978,41717,POINT (445229.308 6471580.655),0.000747,shrub_sphagnum


In [13]:
x_train = train[train.columns[0:14]]
y_train = train[train.columns[17]]

x_test = test[test.columns[0:14]]
y_test = test[test.columns[17]]

x_test.head(2)

,band62,band63,band64,band65,band66,band67,band68,band69,band70,band71,band72,band73,band74,band75
92,0.077456,0.085615,0.095159,0.108089,0.121101,0.133767,0.145966,0.156868,0.164285,0.174290,0.183721,0.190560,0.196722,0.199273
96,0.069943,0.076910,0.085946,0.098197,0.110385,0.122526,0.132732,0.144744,0.155094,0.165884,0.173453,0.178743,0.184820,0.189354


In [14]:
from sklearn import dummy

dc = dummy.DummyClassifier()
dc.fit(x_train, y_train)
dc_score = dc.score(x_test, y_test)

print("Baseline Accuracy:", dc_score)


Baseline Accuracy: 0.16666666666666666


In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(max_depth=2, random_state=42)
# fit to the training data
clf.fit(x_train, y_train)
print(clf.score(x_train, y_train))

# print(clf.predict(x_test))


# predict using the model
clf_train_pred = clf.predict(x_train)
clf_pred = clf.predict(x_test)

# append to the results dataframe
# model_results = model_results.assign(rf_pred = rf_pred)
# print(model_results)

# test the performance of the model
scores = cross_val_score(clf, x_train, y_train, cv=3)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.7380952380952381
0.58 accuracy with a standard deviation of 0.02


In [16]:
test_score = cross_val_score(clf, x_test, y_test, cv=3)
print("%0.2f accuracy with a standard deviation of %0.2f" % (test_score.mean(), test_score.std()))

0.56 accuracy with a standard deviation of 0.08


In [17]:
model_results = test[test.columns[15:18]]

model_results.head()

,geometry,GRID_CODE,PFT
92,POINT (445109.308 6472315.655),0.001311,shrub_sphagnum
96,POINT (445204.308 6472180.655),0.001153,shrub_sphagnum
90,POINT (445129.308 6472370.655),0.001516,shrub_sphagnum
115,POINT (445134.308 6471655.655),0.000281,shrub_sphagnum
112,POINT (445334.308 6471750.655),0.000631,shrub_sphagnum


In [18]:
train_focused = train[train.columns[15:18]]

train_focused.head()

,geometry,GRID_CODE,PFT
99,POINT (444999.308 6472130.655),0.001566,shrub_sphagnum
105,POINT (445034.308 6472000.655),0.000598,shrub_sphagnum
95,POINT (445049.308 6472265.655),0.001928,shrub_sphagnum
103,POINT (445269.308 6472020.655),0.001278,shrub_sphagnum
117,POINT (445229.308 6471580.655),0.000747,shrub_sphagnum


In [19]:
model_results.to_file("cross_lochs_results_sample_RE_gdf.shp")

In [20]:
train_focused.to_file("cross_lochs_train_sample_RE_gdf.shp")

In [21]:
model_results.to_csv("cross_lochs_results_sample_RE_gdf.csv")
train_focused.to_csv("cross_lochs_train_sample_RE_gdf.csv")

In [22]:
# now want to predict for the whole dataset using the random forest
cross_lochs = pd.read_csv('../cross_lochs_gdf.csv')

cross_lochs['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs['geometry'])

cross_lochs_gdf = geopandas.GeoDataFrame(cross_lochs, geometry='geometry')

cross_lochs_gdf

,Unnamed: 0,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,band353,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT
0,0,0.001099,0.003430,0.004752,0.006146,0.006639,0.009240,0.010558,0.011710,0.013801,...,0.074587,0.072792,0.075150,0.074088,0.074698,0.072248,1,POINT (444724.308 6472605.655),0.001099,NaN
1,1,0.003365,0.002541,0.004668,0.006236,0.006519,0.008654,0.010603,0.012083,0.012975,...,0.069210,0.070987,0.072956,0.070970,0.067680,0.067113,2,POINT (444729.308 6472605.655),0.003365,NaN
2,2,0.002603,0.002340,0.005392,0.006031,0.006805,0.008977,0.010990,0.012230,0.014469,...,0.086315,0.086899,0.087825,0.089891,0.086141,0.085678,3,POINT (444734.308 6472605.655),0.002603,NaN
3,3,0.002381,0.004279,0.005815,0.006092,0.007620,0.009825,0.012374,0.014953,0.016326,...,0.103626,0.101852,0.101284,0.103357,0.100732,0.099883,4,POINT (444739.308 6472605.655),0.002381,NaN
4,4,0.002342,0.002837,0.005055,0.005912,0.006696,0.009762,0.010999,0.012902,0.015368,...,0.093658,0.094130,0.091242,0.095368,0.093193,0.093215,5,POINT (444744.308 6472605.655),0.002342,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42016,42016,0.001845,0.002345,0.004714,0.005430,0.006393,0.008860,0.010772,0.012449,0.014887,...,0.077407,0.076553,0.078615,0.078627,0.078251,0.077265,42017,POINT (445714.308 6471575.655),0.001845,NaN
42017,42017,0.002455,0.002271,0.004029,0.005480,0.006396,0.008863,0.011025,0.012685,0.014688,...,0.073803,0.074018,0.074871,0.075158,0.073604,0.075263,42018,POINT (445719.308 6471575.655),0.002455,NaN
42018,42018,0.002387,0.002279,0.004424,0.005426,0.006352,0.008929,0.010853,0.012708,0.014964,...,0.082753,0.083324,0.083870,0.084588,0.081337,0.082178,42019,POINT (445724.308 6471575.655),0.002387,NaN
42019,42019,0.002777,0.002263,0.004564,0.005324,0.006339,0.008972,0.011173,0.012952,0.015367,...,0.094481,0.092089,0.094319,0.095786,0.090843,0.091052,42020,POINT (445729.308 6471575.655),0.002777,NaN


In [24]:
x = cross_lochs_gdf[cross_lochs_gdf.columns[62:76]]

x.head()

,band62,band63,band64,band65,band66,band67,band68,band69,band70,band71,band72,band73,band74,band75
0,0.056894,0.063847,0.077289,0.095265,0.116054,0.137107,0.160438,0.190008,0.214233,0.239633,0.265513,0.281297,0.297386,0.305840
1,0.052981,0.060437,0.073406,0.089232,0.110960,0.132090,0.155294,0.185010,0.210454,0.236916,0.261402,0.281354,0.295005,0.305940
2,0.070318,0.079338,0.093115,0.109797,0.127360,0.146705,0.168035,0.189431,0.208450,0.230109,0.246149,0.257855,0.269344,0.276454
3,0.083830,0.093169,0.106521,0.122714,0.143032,0.159298,0.179304,0.199284,0.218350,0.237514,0.254830,0.267881,0.276413,0.285536
4,0.072251,0.081557,0.095215,0.112238,0.130422,0.150109,0.169687,0.192265,0.213735,0.234733,0.252688,0.267977,0.277578,0.285995


In [25]:
clf_pred_full = clf.predict(x)

clf_pred_full

array(['sphagnum_r', 'sphagnum_r', 'sphagnum_r', ..., 'calluna_mix',
       'calluna_mix', 'calluna_mix'], dtype=object)

In [26]:
model_results_full = cross_lochs_gdf[cross_lochs_gdf.columns[360:362]]

model_results_full = model_results_full.assign(clf_pred_full = clf_pred_full)

model_results_full.head()

,geometry,GRID_CODE,clf_pred_full
0,POINT (444724.308 6472605.655),0.001099,sphagnum_r
1,POINT (444729.308 6472605.655),0.003365,sphagnum_r
2,POINT (444734.308 6472605.655),0.002603,sphagnum_r
3,POINT (444739.308 6472605.655),0.002381,sphagnum_r
4,POINT (444744.308 6472605.655),0.002342,sphagnum_r


In [27]:
model_results_full.to_file("cross_lochs_results_sample_full_RE_gdf.shp")

model_results_full.to_csv("cross_lochs_results_sample_full_RE_gdf.csv")


/tmp/ipykernel_3593/874757293.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  model_results_full.to_file("cross_lochs_results_sample_full_RE_gdf.shp")
